<a href="https://colab.research.google.com/github/TOKTAK007/Alexnet-with-TOKTAK/blob/main/Alexnet_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import os
import glob
import torch
import torch.nn as nn
import torch.optim as optim
import time
from torch.utils.data import random_split, DataLoader
import matplotlib.pyplot as plt
from PIL import Image
import random

#Step 0: Predefined Parameters.

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

datasets_path = '/Users/user/Desktop/test/input_data/datasets'
train_size = 0.7; val_size = 0.2; test_size = 0.1
seed = 42
batch_size = 32
model_name = 'TOKTAK'

checkpoint_path = '/Users/user/Desktop/test/input_data/train_model/' + model_name + '.ckpt'
output_dim = 2
image_dim_1 = 32; image_dim_2 = 32
epochs = 20
image2plot = 10

#Step 1: Splitting the Dataset and Viewing Images. 

# class_names = os.listdir(datasets_path)
class_names = [f.name for f in os.scandir(datasets_path) if f.is_dir() and not f.name.startswith('.')]
print('class names: ', class_names)
num_class = len(class_names)
image_files=glob.glob(datasets_path + '/*/*.png', recursive=True)

print('total images in: ', datasets_path, ' is ', len(image_files))

idx_to_class = {i:j for i, j in enumerate(class_names)}
class_to_idx = {value:key for key, value in idx_to_class.items()}
train_idx, test_idx, val_idx = random_split(image_files, [train_size, val_size, test_size])
train_list=[image_files[i] for i in train_idx.indices]
val_list=[image_files[i] for i in test_idx.indices]
test_list=[image_files[i] for i in val_idx.indices]

print('number of training images: ', len(train_list),
	'\nnumber of val images: ', len(val_list),
	'\nnumber of test images: ', len(test_list))

def view_images(train_list, num_class):
	num_rows = 5; num_cols = 5
	num_images = num_rows * num_cols
	fig, axes = plt.subplots(num_rows, num_cols)
	displayed_classes = set()
	while len(displayed_classes) < num_class:
		random_images = random.sample(train_list, num_images)
		for i, ax in enumerate(axes.flatten()):
			img_path = random_images[i]
			img = Image.open(img_path)
			class_name = img_path.split('/')[-2]
			ax.imshow(img)
			ax.set_title(f'{class_name}')
			ax.axis('off')
			displayed_classes.add(class_name)
	plt.tight_layout()
	plt.show()

# view_images(train_list, num_class)


#Step 2: Data Preprocessing

from sklearn.preprocessing import StandardScaler
from skimage import io
import numpy as np

def means_std(train_list):
	train_data = []
	for img_path in train_list:
		img = io.imread(img_path)
		img = img / 255.0  # limit value to be between 0 and 1
		train_data.append(img)
	train_data = np.array(train_data)
	train_data = np.transpose(train_data, (0, 3, 1, 2))  # fit PyTorch format
	train_data_flat = train_data.reshape(train_data.shape[0], -1)
	scaler = StandardScaler()
	scaler.fit(train_data_flat)
	mean = scaler.mean_.reshape(3, -1).mean(axis=1)
	std = scaler.scale_.reshape(3, -1).std(axis=1)
	print('mean: ', mean)
	print('standard deviation: ', std)
	return mean, std

from torchvision import transforms

class SatelliteDataset():
	def __init__(self, image_paths, class_to_idx, transform=None):
		self.image_paths = image_paths
		self.class_to_idx = class_to_idx
		self.transform = transform

	def __len__(self):
		return len(self.image_paths)

	def __getitem__(self, idx):
		image_filepath = self.image_paths[idx]
		image = Image.open(image_filepath)
		label = image_filepath.split('/')[-2]
		label = self.class_to_idx[label]
		if self.transform is not None:
			image = self.transform(image)
		return image, label

# mean, std = means_std(train_list)

mean = [0.27140397, 0.28222303, 0.23752819]
std = [0.00029074, 0.00028769, 0.00028466]

flip = transforms.RandomHorizontalFlip()
to_tensor = transforms.ToTensor()
normalize = transforms.Normalize(mean, std)
transform_train = transforms.Compose([flip, to_tensor, normalize])
transform_val = transforms.Compose([to_tensor, normalize])
transform_test = transforms.Compose([to_tensor, normalize])
train_dataset = SatelliteDataset(train_list, class_to_idx, transform_train)
val_dataset = SatelliteDataset(val_list, class_to_idx, transform_val)
test_dataset = SatelliteDataset(test_list, class_to_idx, transform_test)
train_loader = DataLoader(train_dataset, batch_size=batch_size,
    shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size,
    shuffle=False, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size,
    shuffle=False, drop_last=False)

# view_images(train_loader, num_class, class_names, mean=mean, std=std)

#Step 3: Model Initialization and Setup. 

class AlexNet(nn.Module):
	def __init__(self, output_dim):
		super().__init__()
		self.features = nn.Sequential(
			nn.Conv2d(3, 64, 3, 2, 1),  # in_channels, out_channels, kernel_size, stride, padding
			nn.MaxPool2d(2),  # kernel_size
			nn.ReLU(inplace=True),
			nn.Conv2d(64, 192, 3, padding=1),
			nn.MaxPool2d(2),
			nn.ReLU(inplace=True),
			nn.Conv2d(192, 384, 3, padding=1),
			nn.ReLU(inplace=True),
			nn.Conv2d(384, 256, 3, padding=1),
			nn.ReLU(inplace=True),
			nn.Conv2d(256, 256, 3, padding=1),
			nn.MaxPool2d(2),
			nn.ReLU(inplace=True)
		)
		self.classifier = nn.Sequential(
			nn.Dropout(0.5),
			nn.Linear(256 * 2 * 2, 4096),
			nn.ReLU(inplace=True),
			nn.Dropout(0.5),
			nn.Linear(4096, 4096),
			nn.ReLU(inplace=True),
			nn.Linear(4096, output_dim),
		)

	def forward(self, x):
		x = self.features(x)
		h = x.view(x.shape[0], -1)
		x = self.classifier(h)
		return x, h

from torchsummary import summary

model = AlexNet(output_dim)

summary(model, (3, image_dim_1, image_dim_2))
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

#Step 4: Training the Model.
from tqdm import tqdm

def calculate_accuracy(y_pred, y):
	top_pred = y_pred.argmax(1, keepdim=True)
	correct = top_pred.eq(y.view_as(top_pred)).sum()
	acc = correct.float() / y.shape[0]
	return acc

def train(model, iterator, optimizer, criterion, device):
	epoch_loss = 0; epoch_acc = 0
	model.train()
	for (x, y) in tqdm(iterator, desc='Training', leave=False):
		x = x.to(device)
		y = y.to(device)
		optimizer.zero_grad()
		y_pred, _ = model(x)
		loss = criterion(y_pred, y)
		acc = calculate_accuracy(y_pred, y)
		loss.backward()
		optimizer.step()
		epoch_loss += loss.item()
		epoch_acc += acc.item()
	return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion, device):
	epoch_loss = 0; epoch_acc = 0
	model.eval()
	with torch.no_grad():
		for (x, y) in tqdm(iterator, desc='Evaluating', leave=False):
			x = x.to(device)
			y = y.to(device)
			y_pred, _ = model(x)
			loss = criterion(y_pred, y)
			acc = calculate_accuracy(y_pred, y)
			epoch_loss += loss.item()
			epoch_acc += acc.item()
	return epoch_loss / len(iterator), epoch_acc / len(iterator)

def epoch_time(start_time, end_time):
	elapsed_time = end_time - start_time
	elapsed_mins = int(elapsed_time / 60)
	elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
	return elapsed_mins, elapsed_secs

def loss_history_plot(history_train, history_valid, model_name):
	axis_x = np.linspace(0, len(history_train), len(history_train))
	plt.plot(axis_x, history_train, linestyle='solid',
			 color='red', linewidth=1, marker='o', ms=5, label='train')
	plt.plot(axis_x, history_valid, linestyle='solid',
			 color='blue', linewidth=1, marker='o', ms=5, label='valid')
	plt.xlabel('epoch')
	plt.ylabel('loss')
	plt.legend(['train', 'valid'])
	plt.title(model_name + ': ' + 'Accuracy', fontweight='bold')
	# plt.savefig('data_out/' + 'resnet' + '.svg', format='svg', bbox_inches='tight', transparent=True, pad_inches=0)
	plt.show()


history_train_loss = []
history_valid_loss = []
best_valid_loss = float('inf')
for epoch in range(epochs):
    start_time = time.monotonic()
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, val_loader, criterion, device)
    history_train_loss.append(train_loss)
    history_valid_loss.append(valid_loss)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), checkpoint_path)
    end_time = time.monotonic()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
loss_history_plot(history_train_loss, history_valid_loss, model_name)

#Step 5: Plotting Confusion Matrix.
from sklearn import metrics

def plot_confusion_matrix_CIFAR10(labels, pred_labels, classes):
	fig = plt.figure()
	ax = fig.add_subplot(1, 1, 1)
	cm = metrics.confusion_matrix(labels, pred_labels)
	cm = metrics.ConfusionMatrixDisplay(cm, display_labels=classes)
	cm.plot(values_format='d', cmap='Greens', ax=ax)
	# plt.savefig('data_out/' + 'CM_resnet' + '.svg', format='svg', bbox_inches='tight', transparent=True, pad_inches=0)
	plt.show()

import torch.nn.functional as TF

def get_predictions(model, iterator, device):
	model.eval()
	images = []; labels = []; probs = []
	with torch.no_grad():
		for (x, y) in iterator:
			x = x.to(device)
			y_pred, _ = model(x)
			y_prob = TF.softmax(y_pred, dim=-1)
			images.append(x.cpu())
			labels.append(y.cpu())
			probs.append(y_prob.cpu())
	images = torch.cat(images, dim=0)
	labels = torch.cat(labels, dim=0)
	probs = torch.cat(probs, dim=0)
	return images, labels, probs

model.load_state_dict(torch.load(checkpoint_path))
test_loss, test_acc = evaluate(model, test_loader, criterion, device)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')
images, labels, probs = get_predictions(model, test_loader, device)
pred_labels = torch.argmax(probs, 1)
plot_confusion_matrix_CIFAR10(labels, pred_labels, class_names)

#Step 6: Plotting the Most Incorrect Prediction.

def normalize_image(image):
	image_min = image.min()
	image_max = image.max()
	image.clamp_(min=image_min, max=image_max)
	image.add_(-image_min).div_(image_max - image_min + 1e-5)
	return image

def plot_most_incorrect_CIFAR10(incorrect, classes, n_images, normalize=True):
	rows = int(np.sqrt(n_images))
	cols = int(np.sqrt(n_images))
	fig = plt.figure()
	for i in range(rows*cols):
		ax = fig.add_subplot(rows, cols, i+1)
		image, true_label, probs = incorrect[i]
		image = image.permute(1, 2, 0)
		true_prob = probs[true_label]
		incorrect_prob, incorrect_label = torch.max(probs, dim=0)
		true_class = classes[true_label]
		incorrect_class = classes[incorrect_label]
		if normalize:
			image = normalize_image(image)
		ax.imshow(image.cpu().numpy())
		ax.set_title(f'true label: {true_class} ({true_prob:.3f})\n'
             f'pred label: {incorrect_class} ({incorrect_prob:.3f})',
             fontsize=6)
		ax.axis('off')
	fig.subplots_adjust(hspace=0.6)
	# plt.savefig('data_out/' + 'incorrect_resnet' + '.svg', format='svg', bbox_inches='tight', transparent=True, pad_inches=0)
	plt.show()

corrects = torch.eq(labels, pred_labels)
incorrect_examples = []
for image, label, prob, correct in zip(images, labels, probs, corrects):
    if not correct:
        incorrect_examples.append((image, label, prob))
incorrect_examples.sort(reverse = True, key=lambda x: torch.max(x[2], dim=0).values)
if len(incorrect_examples) >= image2plot:
	plot_most_incorrect_CIFAR10(incorrect_examples, class_names, image2plot)
else:
    print('reduce the number of image2plot')


